In [2]:
import pandas as pd
from collections import Counter
import datetime
import numpy as np
%matplotlib inline

In [3]:
from sklearn import preprocessing
from sklearn.metrics import log_loss

In [4]:
df = pd.read_csv('../Output/train_step3.csv') # 数据加载
test_data = pd.read_csv('../Output/test_step3.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477334 entries, 0 to 477333
Data columns (total 39 columns):
item_id                        477334 non-null int64
item_brand_id                  477334 non-null int64
item_city_id                   477334 non-null int64
item_price_level               477334 non-null int64
item_sales_level               477334 non-null int64
item_collected_level           477334 non-null int64
item_pv_level                  477334 non-null int64
user_id                        477334 non-null int64
user_age_level                 477334 non-null int64
user_star_level                477334 non-null int64
context_timestamp              477334 non-null int64
context_page_id                477334 non-null int64
shop_id                        477334 non-null int64
shop_review_num_level          477334 non-null int64
shop_review_positive_rate      477334 non-null float64
shop_star_level                477334 non-null int64
shop_score_service             477334 n

将数据集按照context_timestamp的日期划分成训练集和测试集

In [4]:
ts = [datetime.datetime.fromtimestamp(df['context_timestamp'][i]).day for i in range(df.shape[0])]
Counter(ts)

Counter({18: 78165,
         19: 70790,
         20: 68270,
         21: 71063,
         22: 68186,
         23: 63521,
         24: 57339})

18-23日的当成训练集，24日的当成测试集

In [5]:
df['context_time_day'] = [datetime.datetime.fromtimestamp(df['context_timestamp'][i]).day for i in range(df.shape[0])]

In [6]:
df1 = df.drop(['context_time_day', 'context_timestamp','instance_id'], 1)
test_data = test_data.drop(['context_timestamp'], 1)

In [7]:
df_label = df1.pop('is_trade')
df_data = df1
del df1

In [8]:
instance_df = test_data.pop('instance_id') # pd.series

rows = df_data.shape[0]
merge_data = df_data.append(test_data)

In [8]:
merge_data.info()

In [9]:
scale_data = preprocessing.scale(merge_data)

data = scale_data[0:rows]  # 训练集数据
test_data = scale_data[rows:] # 测试集数据

In [10]:
train_df = data[df['context_time_day']<=23]
test_df = data[df['context_time_day']>23]

In [11]:
train_df_label = df_label[df['context_time_day']<=23]
test_df_label = df_label[df['context_time_day']>23]

In [12]:
train_df

# base_line
LinearRegression线性回归模型

In [ ]:
from sklearn import linear_model

In [11]:
LR_model = linear_model.LinearRegression()

In [12]:
LR_model.fit(train_df, train_df_label)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [13]:
print (LR_model.coef_)
print (LR_model.intercept_)

[  1.64462649e-04  -7.59378476e-04  -4.78603443e-04  -5.22415029e-03
  -5.23812786e-03  -4.06937069e-03  -1.13207445e-04  -1.24624165e-03
   2.86074570e-04  -3.88124243e-03   2.45893100e-04   1.87721684e-03
   3.27046745e-03  -5.85566825e-03  -6.83376042e-04   9.25856840e-05
   1.32941609e-03   1.20688466e-03  -6.47682743e-05   1.31912207e-04
   1.26124384e-03   1.17390892e-03   1.14826464e-03  -2.25812485e-03
  -9.61062655e-04  -5.33575071e+10   2.90834095e+10   7.25369222e+09
   1.37134330e+10   3.04793958e+10  -3.52564911e+10  -9.37618073e+10
  -8.66461067e+10  -3.16663353e+10   3.67095474e+08   2.72704950e+08
   3.74666528e+08   5.22381781e+08   6.96299803e+08   9.21601584e+08
   1.26929953e+09   1.60442442e+09   1.95423050e+09   2.33251276e+09
   2.75688505e+09   3.16451224e+09   3.26546426e+09   2.89967384e+09
   2.45454262e+09   1.77944208e+09   1.28156534e+09   5.44689999e+08
  -8.89488354e+10  -1.02911927e+10  -7.28212108e+09  -2.23990361e+10
  -3.02775923e+10  -2.79046734e+10

In [14]:
test_df_preLabel = LR_model.predict(test_df)

In [15]:
logLoss = log_loss(test_df_label, test_df_preLabel)
print (logLoss)

0.0868520141726


In [16]:
test_prelabel = LR_model.predict(test_data)

In [17]:
result = pd.DataFrame({'instance_id':instance_df})
result['predicted_score'] = test_prelabel

In [18]:
result.to_csv('../Output/predict_result_0321.txt', index=False, sep=' ', line_terminator='\r')